<a href="https://colab.research.google.com/github/cvs2010/rsp/blob/main/notebook_teste_importsheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook criado com intuito de gerar um algoritmo para importação dos dados das planilhas Horario RSP

## Observações:
- Tente o mínimo possível utilizar as bandas de GPU e TPU grátis do colab
- Esse arquivo é um arquivo de histórico, então tente trabalhar no seu GoogleColab em paralelo, somente suba as células para esse Notebook após finalizar todo o código ou gerar um checkpoint de análise do código (momento em que o código ainda não está pronto, mas necessita de análise do grupo)
- Muito cuidado com a edição desse arquivo e de outros arquivos da pasta do Drive: mais uma vez, tente trabalhar em paralelo, nunca utilize os arquivos originais, faça cópia de arquivos e trabalhe com seus arquivos localmente ou via cloud em seu login.

# Configurações Iniciais

## Importação de bibliotecas:
- bibliotecas disponíveis: import scipy, matplotlib, numpy, pandas, sklearn, seaborn, skimage, xgboost, lightgbm, catboost, tensorflow, keras, theano, pydot, graphviz, h5py, cv2, PIL, requests, bs4, lxml, html5lib, pyproj, shapely, geopandas, folium, nltk, spacy, textblob, gensim, glove_python, transformers, sentence_transformers, torchvision, torchtext, spacy, openai, tqdm, watermark, imageio, moviepy, simpleaudio, pygame, bokeh, plotly, dash, dash_core_components, dash_html_components, dash_bootstrap_components, ipywidgets, ipython, autoreload, pprint, inspect, warnings, logging, time, datetime, os, sys, math, random, json, re, hashlib, base64, uuid, collections, itertools, functools

In [469]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import numpy as np
import os
import re
from datetime import datetime
!pip install unidecode
from unidecode import unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Montando GoogleDrive

In [470]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importação do DataFrame

In [471]:
df = pd.read_csv('https://github.com/cvs2010/rsp/raw/main/files/dataframe_backup.csv')

In [472]:
df

,Coluna 1,Coluna 2,Coluna 3,Coluna 4,Coluna 5,Coluna 6,Coluna 7,Coluna 8,Coluna 9,Coluna 10,Coluna 11,Coluna 12,Coluna 13,Coluna 14,Coluna 15,Coluna 16,Coluna 17,Coluna 18,Coluna 19,Coluna 20,Coluna 21,Coluna 22,Coluna 23,Coluna 24,Coluna 25,Coluna 26,Coluna 27,Coluna 28,Coluna 29,Coluna 30,Coluna 31,Coluna 32,Coluna 33,Coluna 34,Coluna 35,Coluna 36,Coluna 37,Coluna 38,Coluna 39,Coluna 40,Coluna 41,Coluna 42,Coluna 43,Coluna 44,Coluna 45,Coluna 46,Coluna 47,Coluna 48,Coluna 49,Coluna 50,Coluna 51,Coluna 52,Coluna 53,Coluna 54,Coluna 55,Coluna 56,Coluna 57,Coluna 58,Coluna 59,Coluna 60,Coluna 61,Coluna 62,Coluna 63,Coluna 64,Coluna 65,Coluna 66,Coluna 67,Coluna 68,Coluna 69,Coluna 70,Coluna 71,Coluna 72,Coluna 73,Coluna 74,Coluna 75,Coluna 76,Coluna 77,Coluna 78,Coluna 79,Coluna 80,Coluna 81,Coluna 82,Coluna 83,Coluna 84,Coluna 85,Coluna 86,Coluna 87,Coluna 88,Coluna 89,Coluna 90,Coluna 91,Coluna 92,Coluna 93,Coluna 94,Coluna 95,Coluna 96,Coluna 97,Coluna 98,Coluna 99,Coluna 100
0,horas,num_traf,dd_mm_aaaa,ds_fs_ff,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,02:30,02:45,03:00,03:15,03:30,03:45,04:00,04:15,04:30,04:45,05:00,05:15,05:30,05:45,06:00,06:15,06:30,06:45,07:00,07:15,07:30,07:45,08:00,08:15,08:30,08:45,09:00,09:15,09:30,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,16:00,16:15,16:30,16:45,17:00,17:15,17:30,17:45,18:00,18:15,18:30,18:45,19:00,19:15,19:30,19:45,20:00,20:15,20:30,20:45,21:00,21:15,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
1,D1Y1,1048,01/01/2017,DOM,4,2,3,2,2,2,2,2,2,2,2,2,2,7,4,10,7,10,4,10,9,8,9,8,2,3,7,7,7,7,5,4,7,10,9,3,7,2,9,2,6,10,3,2,9,7,5,6,3,9,5,4,7,5,9,6,10,10,9,2,9,7,4,3,10,7,4,10,9,4,10,9,5,2,10,5,10,5,4,2,9,7,7,6,2,5,6,2,9,4,9,8,4,4,4,6
2,D2Y1,962,02/01/2017,SEG,3,4,5,2,2,2,2,2,2,2,2,2,2,9,7,8,10,8,8,4,2,4,4,7,2,6,8,9,8,6,8,7,7,8,5,8,7,7,4,10,2,6,4,5,3,6,3,2,7,7,5,8,4,3,5,8,7,6,10,9,10,9,10,6,4,10,5,10,4,9,2,4,5,5,3,10,9,4,2,10,6,6,3,2,5,2,9,9,8,10,9,9,4,3,3,10
3,D3Y1,1150,03/01/2017,TER,2,4,3,2,2,2,2,2,2,2,2,2,2,7,7,5,3,9,10,3,5,3,6,5,5,4,8,10,8,6,8,6,4,10,5,6,8,3,5,7,8,2,8,2,6,5,4,6,8,7,4,6,4,6,6,7,8,3,6,4,2,6,3,5,10,8,8,3,3,3,5,4,4,8,5,3,4,6,3,9,4,8,4,8,6,7,10,8,4,7,7,5,3,3,2,7
4,D4Y1,685,04/01/2017,QUA,3,4,4,2,2,2,2,2,2,2,2,2,2,2,9,5,5,8,7,8,10,7,4,4,10,2,5,2,4,8,2,9,8,6,10,7,5,5,7,7,8,3,7,9,10,7,9,2,4,10,8,4,10,3,7,9,8,6,6,8,10,3,7,6,10,7,2,10,3,2,2,8,4,5,4,8,6,8,7,5,8,3,4,2,10,6,6,4,6,6,8,10,5,9,8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,D361Y5,913,27/12/2023,QUA,2,3,3,2,2,2,2,2,2,2,2,2,2,6,10,7,4,9,8,5,9,7,3,9,3,3,10,4,4,3,8,3,4,4,4,10,8,8,10,8,10,3,3,8,8,8,8,5,8,8,7,9,7,6,5,5,5,5,3,9,4,5,3,2,5,6,10,6,8,10,8,7,8,10,7,5,8,10,10,8,3,10,8,8,7,7,10,5,3,10,5,3,6,3,8,5
1822,D362Y5,1082,28/12/2023,QUI,3,4,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,5,3,4,10,3,8,7,6,5,5,2,10,4,5,5,6,2,4,4,2,4,4,5,10,6,2,6,6,10,3,4,6,7,7,8,7,6,9,2,2,3,4,10,5,3,8,9,7,10,3,4,8,4,2,10,2,2,3,9,8,10,5,4,8,5,3,5,5,7,5,2,9,3,4,4,9,4,2,6
1823,D363Y5,1154,29/12/2023,SEX,5,5,4,2,2,2,2,2,2,2,2,2,2,6,3,6,3,7,9,10,2,3,3,10,10,6,8,3,5,2,6,5,3,2,5,4,10,2,2,5,9,6,10,4,4,6,2,6,6,7,9,8,6,10,6,9,9,10,4,2,3,5,5,2,6,3,4,10,3,10,5,10,4,8,3,6,8,10,8,10,5,6,6,8,5,3,10,4,7,6,6,7,6,2,6,6
1824,D364Y5,963,30/12/2023,SÁB,4,5,3,2,2,2,2,2,2,2,2,2,2,9,10,7,8,10,4,3,2,7,5,5,7,9,3,10,6,3,4,4,10,2,9,6,8,9,3,9,9,7,4,10,4,6,5,4,9,7,6,3,10,9,5,7,10,8,2,7,4,7,2,9,4,3,6,9,8,6,4,7,3,5,8,3,10,9,7,6,4,5,2,7,2,5,10,5,2,8,6,6,2,6,7,3


# Extração dos dados (linha de nº de operadores) de planilhas

## Importando DataSet
- Importando o DataSet HorarioTeste.zip do repositório do github
- Extraindo os arquivos na pasta /content/


In [473]:
# !wget https://github.com/cvs2010/rsp/raw/main/files/HorariosTeste.zip
# !unzip HorariosTeste.zip -d /content/

import urllib.request
import zipfile
import io

url = "https://github.com/cvs2010/rsp/raw/main/files/HorariosTeste.zip"
with urllib.request.urlopen(url) as u:
    f = io.BytesIO(u.read())

with zipfile.ZipFile(f, 'r') as zip_ref:
    zip_ref.extractall('/content/')

## Resgatando a informação de data do nome do arquivo
Após a extração da data e transformação em uma string de data legível, iremos inserir essa informação na linha/coluna de data do dataframe
- O próximo passo é fazer com que ele leia todos os arquivos dentro da pasta HorariosTeste e processar um a um automaticamente de forma ordenada

In [492]:
# Para fazer esse processo são necessárias as bibliotecas os, re, unidecode e datetime

# Caminho do arquivo
filename = '/content/HorariosTeste/Horário RSP - Backup - 12-04-2023 - WED.xlsx'

# Expressão regular para encontrar a data no nome do arquivo
date_regex = r'(\d{1,2})[-_./](\d{1,2})[-_./](\d{4})'

# Encontra a primeira data correspondente no nome do arquivo
match = re.search(date_regex, filename)

# Extrai a data no formato dd-mm-yyyy
def extract_date(filename):
    pattern = r'(\d{2})\s(\w{3})\s(\d{4})'
    match = re.search(pattern, filename)
    if match:
        day, month, year = match.groups()
        months = {
            'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
            'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'
        }
        month = months[month.upper()]
        return f'{day}-{month}-{year}'
    else:
        return ""

# Rodando a função filename
formatted_date = filename
date = extract_date(formatted_date)
print(filename)
print(date)

# Extrai a data no formato dd-mm-yyyy
if match:
    day = match.group(1).zfill(2)
    month = match.group(2).zfill(2)
    year = match.group(3)
    formatted_date = f'{day}-{month}-{year}'
    print(formatted_date)
else:
    print('')

/content/HorariosTeste/Horário RSP - Backup - 12-04-2023 - WED.xlsx

12-04-2023


## Extraindo a linha 'OPERADORES' da 1ª planilha - Horаrio RSP - Backup - 12-04-2023 - WED

In [493]:
# Essa célula necessita da biblioteca pandas importada

# Caminho para o arquivo Excel
file_path = '/content/HorariosTeste/Horário RSP - Backup - 12-04-2023 - WED.xlsx'

# Ler a planilha "MANHÃ" para um DataFrame (df) do Pandas
planilha1 = pd.read_excel(file_path, sheet_name='MANHÃ')

# Mostrar as últimas 20 linhas da Planilha
planilha1

,DATA,Unnamed: 1,Unnamed: 2,REGIÃO SP - MANHÃ,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,SUPERVISOR,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,ÁREA RESTRITA PARA OS DESENVOLVEDORES,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78
0,13/04/2023 - QUINTA-FEIRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HORA ATUAL,NaN,NaN,6,NaN,NaN,NaN,7,NaN,NaN,NaN,8,NaN,NaN,NaN,9,NaN,NaN,NaN,10,NaN,NaN,NaN,11,NaN,NaN,NaN,12,NaN,NaN,NaN,13,NaN,NaN,NaN,14,NaN,NaN,NaN,15,NaN,NaN,NaN,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,suporteplanilhasc1@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-04-13 04:26:17.088000,NaN,NaN,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,2S NILSILENE (I) - SUPERVISOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,06:00,06:15,06:30,06:45,07:00,07:15,07:30,07:45,08:00,08:15,08:30,08:45,09:00,09:15,09:30,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,16:00,16:15,16:30,16:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,2023-04-13 00:00:00,2023-04-13 06:00:00,2023-04-13 06:15:00.288000,2023-04-13 06:29:59.712000,2023-04-13 06:45:00,2023-04-13 07:00:00.288000,2023-04-13 07:14:59.712000,2023-04-13 07:30:00,2023-04-13 07:45:00.288000,2023-04-13 07:59:59.712000,2023-04-13 08:15:00,2023-04-13 08:30:00.288000,2023-04-13 08:44:59.712000,2023-04-13 09:00:00,2023-04-13 09:15:00.288000,2023-04-13 09:29:59.712000,2023-04-13 09:45:00,2023-04-13 10:00:00.288000,2023-04-13 10:14:59.712000,2023-04-13 10:30:00,2023-04-13 10:45:00.288000,2023-04-13 10:59:59.712000,2023-04-13 11:15:00,2023-04-13 11:30:00.288000,2023-04-13 11:44:59.712000,2023-04-13 12:00:00,2023-04-13 12:15:00.288000,2023-04-13 12:29:59.712000,2023-04-13 12:45:00,2023-04-13 13:00:00.288000,2023-04-13 13:14:59.712000,2023-04-13 13:30:00,2023-04-13 13:45:00.288000,2023-04-13 13:59:59.712000,2023-04-13 14:15:00,2023-04-13 14:30:00.288000,2023-04-13 14:44:59.712000,2023-04-13 15:00:00,2023-04-13 15:15:00.288000,2023-04-13 15:29:59.712000,2023-04-13 15:45:00,2023-04-13 16:00:00.288000,2023-04-13 16:14:59.712000,2023-04-13 16:30:00,2023-04-13 16:45:00.288000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2023-04-13 04:26:17.088000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04

In [494]:
planilha1 = planilha1.fillna('')

In [495]:
pd.set_option('display.max_columns', None)

In [496]:
planilha1

,DATA,Unnamed: 1,Unnamed: 2,REGIÃO SP - MANHÃ,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,SUPERVISOR,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,ÁREA RESTRITA PARA OS DESENVOLVEDORES,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78
0,13/04/2023 - QUINTA-FEIRA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,HORA ATUAL,,,6,,,,7,,,,8,,,,9,,,,10,,,,11,,,,12,,,,13,,,,14,,,,15,,,,16,,,,,,,,,suporteplanilhasc1@gmail.com,,,,,,,,,,,,,,,,,,,,,,,,,,
2,2023-04-13 04:26:17.088000,,,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,0,15,30,45,2S NILSILENE (I) - SUPERVISOR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,,,,06:00,06:15,06:30,06:45,07:00,07:15,07:30,07:45,08:00,08:15,08:30,08:45,09:00,09:15,09:30,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,16:00,16:15,16:30,16:45,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,,,2023-04-13 00:00:00,2023-04-13 06:00:00,2023-04-13 06:15:00.288000,2023-04-13 06:29:59.712000,2023-04-13 06:45:00,2023-04-13 07:00:00.288000,2023-04-13 07:14:59.712000,2023-04-13 07:30:00,2023-04-13 07:45:00.288000,2023-04-13 07:59:59.712000,2023-04-13 08:15:00,2023-04-13 08:30:00.288000,2023-04-13 08:44:59.712000,2023-04-13 09:00:00,2023-04-13 09:15:00.288000,2023-04-13 09:29:59.712000,2023-04-13 09:45:00,2023-04-13 10:00:00.288000,2023-04-13 10:14:59.712000,2023-04-13 10:30:00,2023-04-13 10:45:00.288000,2023-04-13 10:59:59.712000,2023-04-13 11:15:00,2023-04-13 11:30:00.288000,2023-04-13 11:44:59.712000,2023-04-13 12:00:00,2023-04-13 12:15:00.288000,2023-04-13 12:29:59.712000,2023-04-13 12:45:00,2023-04-13 13:00:00.288000,2023-04-13 13:14:59.712000,2023-04-13 13:30:00,2023-04-13 13:45:00.288000,2023-04-13 13:59:59.712000,2023-04-13 14:15:00,2023-04-13 14:30:00.288000,2023-04-13 14:44:59.712000,2023-04-13 15:00:00,2023-04-13 15:15:00.288000,2023-04-13 15:29:59.712000,2023-04-13 15:45:00,2023-04-13 16:00:00.288000,2023-04-13 16:14:59.712000,2023-04-13 16:30:00,2023-04-13 16:45:00.288000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,,,2023-04-13 04:26:17.088000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2023-04-13 04:23:16.512000,2

In [497]:
# Lê a planilha e carrega em um dataframe, limpando os dados NaN
planilha1 = pd.read_excel('/content/HorariosTeste/Horário RSP - Backup - 12-04-2023 - WED.xlsx')
planilha1 = planilha1.fillna('')

# Encontra o índice da segunda ocorrência da palavra "OPERADORES"
linha_operadores2 = planilha1.index[planilha1.iloc[:, 0] == 'OPERADORES'][1]

# Busca a linha índice correspondente à linha 40
linha_operadores = planilha1.iloc[linha_operadores2]

# Imprime a linha inteira correspondente à linha 40
linha_operadores

DATA                 OPERADORES
Unnamed: 1                     
Unnamed: 2                     
REGIÃO SP - MANHÃ             0
Unnamed: 4                    0
                        ...    
Unnamed: 74                    
Unnamed: 75                    
Unnamed: 76                    
Unnamed: 77                    
Unnamed: 78                    
Name: 38, Length: 79, dtype: object

Encontrando a linha de horas na planilha

In [498]:
# Lê a planilha e carrega em um dataframe, limpando os dados NaN
planilha1 = pd.read_excel('/content/HorariosTeste/Horário RSP - Backup - 12-04-2023 - WED.xlsx')
planilha1 = planilha1.fillna('')

# Encontra o índice da linha que contém o valor 06:00
linha_hrs_indices = planilha1.index[planilha1.iloc[:, 3] == '06:00']

# Busca a linha índice 06:00 e mostra o resultado
linha_hrs = planilha1.loc[linha_hrs_indices[0]]
linha_hrs

DATA                      
Unnamed: 1                
Unnamed: 2                
REGIÃO SP - MANHÃ    06:00
Unnamed: 4           06:15
                     ...  
Unnamed: 74               
Unnamed: 75               
Unnamed: 76               
Unnamed: 77               
Unnamed: 78               
Name: 3, Length: 79, dtype: object

In [499]:
# Concatena as colunas dos dataframes
linha_op_hrs = pd.concat([linha_operadores, linha_hrs], axis=1)

# Mostra o resultado
linha_op_hrs

,38,3
DATA,OPERADORES,
Unnamed: 1,,
Unnamed: 2,,
REGIÃO SP - MANHÃ,0,06:00
Unnamed: 4,0,06:15
...,...,...
Unnamed: 74,,
Unnamed: 75,,
Unnamed: 76,,
Unnamed: 77,,


In [500]:
# Cria uma nova coluna com o número índice
linha_op_hrs = linha_op_hrs.reset_index()

# Renomeia a coluna "index" para "número índice"
linha_op_hrs = linha_op_hrs.rename(columns={"index": "número índice"})

# Mostra o resultado
linha_op_hrs

,número índice,38,3
0,DATA,OPERADORES,
1,Unnamed: 1,,
2,Unnamed: 2,,
3,REGIÃO SP - MANHÃ,0,06:00
4,Unnamed: 4,0,06:15
...,...,...,...
74,Unnamed: 74,,
75,Unnamed: 75,,
76,Unnamed: 76,,
77,Unnamed: 77,,


In [501]:
# Renomeia as colunas
linha_op_hrs = linha_op_hrs.rename(columns={0: '', linha_op_hrs.columns[-1]: 'horas'})
linha_op_hrs = linha_op_hrs.rename({linha_op_hrs.columns[1]: formatted_date}, axis=1)

# Remove a primeira linha do dataframe
linha_op_hrs = linha_op_hrs.iloc[1:]

# Reseta o índice do dataframe
linha_op_hrs = linha_op_hrs.reset_index(drop=True)

# Deletando a primeira coluna do dataframe linha_op_hrs
linha_op_hrs = linha_op_hrs.drop(labels='número índice', axis=1)

# Mostra o resultado
linha_op_hrs

,12-04-2023,horas
0,,
1,,
2,0,06:00
3,0,06:15
4,0,06:30
...,...,...
73,,
74,,
75,,
76,,


In [502]:
# Limpando os valores nulos
linha_op_hrs = linha_op_hrs.query('horas != ""')

linha_op_hrs

,12-04-2023,horas
2,0,06:00
3,0,06:15
4,0,06:30
5,6,06:45
6,6,07:00
7,6,07:15
8,6,07:30
9,6,07:45
10,5,08:00
11,5,08:15


In [503]:
# Aplicação do transpose
linha_op_hrs_transpose = linha_op_hrs.transpose()
#Cria um arquivo csv para checar se está tudo ok
# linha_op_hrs_transpose.to_csv('linha_op_hrs_transpose.csv', index=False)
linha_op_hrs_transpose

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
12-04-2023,0,0,0,6,6,6,6,6,5,5,6,6,5,8,7,5,5,5,6,6,6,5,5,5,5,4,4,4,5,5,5,4,4,0,0,0,0,0,0,0,0,0,0,0
horas,06:00,06:15,06:30,06:45,07:00,07:15,07:30,07:45,08:00,08:15,08:30,08:45,09:00,09:15,09:30,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,16:00,16:15,16:30,16:45


In [504]:
df.head(2)

,Coluna 1,Coluna 2,Coluna 3,Coluna 4,Coluna 5,Coluna 6,Coluna 7,Coluna 8,Coluna 9,Coluna 10,Coluna 11,Coluna 12,Coluna 13,Coluna 14,Coluna 15,Coluna 16,Coluna 17,Coluna 18,Coluna 19,Coluna 20,Coluna 21,Coluna 22,Coluna 23,Coluna 24,Coluna 25,Coluna 26,Coluna 27,Coluna 28,Coluna 29,Coluna 30,Coluna 31,Coluna 32,Coluna 33,Coluna 34,Coluna 35,Coluna 36,Coluna 37,Coluna 38,Coluna 39,Coluna 40,Coluna 41,Coluna 42,Coluna 43,Coluna 44,Coluna 45,Coluna 46,Coluna 47,Coluna 48,Coluna 49,Coluna 50,Coluna 51,Coluna 52,Coluna 53,Coluna 54,Coluna 55,Coluna 56,Coluna 57,Coluna 58,Coluna 59,Coluna 60,Coluna 61,Coluna 62,Coluna 63,Coluna 64,Coluna 65,Coluna 66,Coluna 67,Coluna 68,Coluna 69,Coluna 70,Coluna 71,Coluna 72,Coluna 73,Coluna 74,Coluna 75,Coluna 76,Coluna 77,Coluna 78,Coluna 79,Coluna 80,Coluna 81,Coluna 82,Coluna 83,Coluna 84,Coluna 85,Coluna 86,Coluna 87,Coluna 88,Coluna 89,Coluna 90,Coluna 91,Coluna 92,Coluna 93,Coluna 94,Coluna 95,Coluna 96,Coluna 97,Coluna 98,Coluna 99,Coluna 100
0,horas,num_traf,dd_mm_aaaa,ds_fs_ff,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,02:30,02:45,03:00,03:15,03:30,03:45,04:00,04:15,04:30,04:45,05:00,05:15,05:30,05:45,06:00,06:15,06:30,06:45,07:00,07:15,07:30,07:45,08:00,08:15,08:30,08:45,09:00,09:15,09:30,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,16:00,16:15,16:30,16:45,17:00,17:15,17:30,17:45,18:00,18:15,18:30,18:45,19:00,19:15,19:30,19:45,20:00,20:15,20:30,20:45,21:00,21:15,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
1,D1Y1,1048,01/01/2017,DOM,4,2,3,2,2,2,2,2,2,2,2,2,2,7,4,10,7,10,4,10,9,8,9,8,2,3,7,7,7,7,5,4,7,10,9,3,7,2,9,2,6,10,3,2,9,7,5,6,3,9,5,4,7,5,9,6,10,10,9,2,9,7,4,3,10,7,4,10,9,4,10,9,5,2,10,5,10,5,4,2,9,7,7,6,2,5,6,2,9,4,9,8,4,4,4,6


# Fazendo um merge de df_test com linha_op_hrs_transpose

In [505]:
# Montando um df_test para não utilizar o df original
df_test = df.head(2)

# Excluindo colunas 1 a 28 e 73 a 100 do df_test
cols_to_remove = np.r_[1:28, 72:100]
df_test = df_test.drop(df_test.columns[cols_to_remove], axis=1)

# Transpondo e mostrando o resultado
df_test = df_test.transpose()

# Removendo a coluna 0
df_test = df_test.reset_index(drop=True)

# Mudando o index das colunas
# Definindo a primeira linha como rótulos das colunas
df_test = df_test.set_axis(df_test.iloc[0], axis=1)

# Excluindo a primeira linha
df_test = df_test.iloc[1:]

# Definindo o index da coluna 1 de df_test como a coluna horas
df_test = df_test.set_index('horas')

# Mostrando o resultado
df_test.head()

,D1Y1
horas,
06:00,2
06:15,3
06:30,7
06:45,7
07:00,7


In [506]:
# Definindo o index da coluna 1 de linha_op_hrs como a coluna horas 
linha_op_hrs = linha_op_hrs.set_index('horas')
linha_op_hrs.head()

,12-04-2023
horas,
06:00,0
06:15,0
06:30,0
06:45,6
07:00,6


In [507]:
# Criando a variável col_name para resgatar o valor da linha 0 de linha_op_hrs
# para colocar esse valor col_name como rótulo da coluna em df_test
col_name = linha_op_hrs.columns[0]

# Inserindo a coluna 0 de linha_op_hrs na posição 2 em df_test
df_test.insert(loc=1, column=col_name, value=linha_op_hrs.iloc[:, 0])

df_test.head()

,D1Y1,12-04-2023
horas,,
06:00,2,0
06:15,3,0
06:30,7,0
06:45,7,6
07:00,7,6


In [508]:
df_test.transpose().head()

horas,06:00,06:15,06:30,06:45,07:00,07:15,07:30,07:45,08:00,08:15,08:30,08:45,09:00,09:15,09:30,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,16:00,16:15,16:30,16:45
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
D1Y1,2,3,7,7,7,7,5,4,7,10,9,3,7,2,9,2,6,10,3,2,9,7,5,6,3,9,5,4,7,5,9,6,10,10,9,2,9,7,4,3,10,7,4,10
12-04-2023,0,0,0,6,6,6,6,6,5,5,6,6,5,8,7,5,5,5,6,6,6,5,5,5,5,4,4,4,5,5,5,4,4,0,0,0,0,0,0,0,0,0,0,0
